### Imports

In [1]:
# Fix imports
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

In [2]:
import collections
import concurrent.futures
import datetime
import json
import termcolor

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

### Setup

In [3]:
# Setup LLM
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    model_name='llama2:13b'
    # streaming=True
)

# Setup measurements and clock
measurements = measurements_lib.Measurements()
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])

# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

# Importance model
importance_model = importance_function.ConstantImportanceModel()

# Build memory
blank_memory_factory = blank_memories.MemoryFactory(
  model=model,
  embedder=embedder,
  importance=importance_model.importance,
  clock_now=clock.now
)

memory = blank_memory_factory.make_blank_memory()

In [4]:
from examples.custom_components.categories import CategoryConcept, sample_statements


# Set configuration for agent and experiment configuration.
agent_config = {
  'name': 'Alice',
  'gender': 'female',
}
experiment_config = {
  'condition': 'generic'
}
with open('generic_statements.json', 'r') as file:
  experiment = json.load(file)

with open('generic_questions.json', 'r') as file:
  questions = json.load(file)

# Develop context
context = (
  f'This is an interaction between two characters, {agent_config["name"]} and '
  f'a psychology researcher who is conducting an experiment. {agent_config["name"]} '
  f'will be asked a series of questions during the experiment, and will give '
  f'an answer to the question according to what {"she" if agent_config["gender"] == "female" else "he"} '
  f'believes is the correct answer to the question. '
  f'\n'
  f'The experimenter begins the experiment by telling {agent_config["name"]}, '
  f'"I would like to tell you something today about animals named Zarpies. Zarpies '
  f'are an animal that lives on an alien planet. Look, this is a Zarpie right here."'
  f'\n'
  f'\n'
  f'The experimenter shows {agent_config["name"]} a picture of a Zarpie, then continues. '
  f'"Now, I am going to tell you a few things that are true about '
  f'{experiment[experiment_config["condition"]]["intro"]}:'
  f'\n'
  f'\n'
  f'{sample_statements(condition=experiment_config["condition"], experiment=experiment, num_statements=5)}'
  f'\n'
  f'\n'
  f'Next, the experimenter presents {agent_config["name"]} with several multiple-choice questions. '
  f'{agent_config["name"]} will choose the answer corresponding to what {"she" if agent_config["gender"] == "female" else "he"} '
  f'believes is the correct answer, beginning {"her" if agent_config["gender"] == "female" else "his"} '
  f'answer with a single choice. If {agent_config["name"]} does not know the answer, '
  f'{"she" if agent_config["gender"] == "female" else "he"} will still guess and '
  f'will still guess and begin the answer with a single choice.'
  f'\n'
  f'\n'
)

# Add the memory and advance the clock
memory.add(
  text=context,
  timestamp=clock.now
)

# Build the agent and its components with the memory
categories = CategoryConcept(
  name="category_concept",
  model=model,
  memory=memory,
  agent_name=agent_config["name"],
  mem_num=1,
  question_bank=questions,
  verbose=True
)

agent = basic_agent.BasicAgent(
  model=model,
  memory=memory,
  agent_name=agent_config["name"],
  clock=clock,
  components=[categories],
  num_memories_retrieved=1,
  verbose=True
)

# Advance the clock
clock.advance()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
print(termcolor.colored(context, 'red'), end = '')

agent.observe(context)

This is an interaction between two characters, Alice and a psychology researcher who is conducting an experiment. Alice will be asked a series of questions during the experiment, and will give an answer to the question according to what she believes is the correct answer to the question. 
The experimenter begins the experiment by telling Alice, "I would like to tell you something today about animals named Zarpies. Zarpies are an animal that lives on an alien planet. Look, this is a Zarpie right here."

The experimenter shows Alice a picture of a Zarpie, then continues. "Now, I am going to tell you a few things that are true about Zarpies:

* Zarpies have a star behind their horns.
* Zarpies hate walking in snow.
* Zarpies stand on their back legs when they are surprised.
* Zarpies have stripes on the bottom of their feet.
* Zarpies are scared of ladybugs.

Next, the experimenter presents Alice with several multiple-choice questions. Alice will choose the answer corresponding to what sh